In [9]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [41]:
input_name =  'Final_PremierQatar_2_21'#'punto_qatar'# 
path = '/content/gdrive/MyDrive/PadelData/'
path_res = '/content/gdrive/MyDrive/PadelRes/'
point_n = 0



# YOLOv5

*Author: Ultralytics*

**YOLOv5 in PyTorch > ONNX > CoreML > TFLite**

_ | _
- | -
![alt](https://pytorch.org/assets/images/ultralytics_yolov5_img1.jpg) | ![alt](https://pytorch.org/assets/images/ultralytics_yolov5_img2.png)


## Before You Start

Start from a **Python>=3.8** environment with **PyTorch>=1.7** installed. To install PyTorch see [https://pytorch.org/get-started/locally/](https://pytorch.org/get-started/locally/). To install YOLOv5 dependencies:

In [1]:
%%bash
pip install -qr https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt  # install dependencies

## Model Description

<img width="800" alt="YOLOv5 Model Comparison" src="https://github.com/ultralytics/yolov5/releases/download/v1.0/model_comparison.png">
&nbsp;

[YOLOv5](https://ultralytics.com/yolov5) 🚀 is a family of compound-scaled object detection models trained on the COCO dataset, and includes simple functionality for Test Time Augmentation (TTA), model ensembling, hyperparameter evolution, and export to ONNX, CoreML and TFLite.

|Model |size<br><sup>(pixels) |mAP<sup>val<br>0.5:0.95 |mAP<sup>test<br>0.5:0.95 |mAP<sup>val<br>0.5 |Speed<br><sup>V100 (ms) | |params<br><sup>(M) |FLOPS<br><sup>640 (B)
|---   |---  |---        |---         |---             |---                |---|---              |---
|[YOLOv5s6](https://github.com/ultralytics/yolov5/releases)   |1280 |43.3     |43.3     |61.9     |**4.3** | |12.7  |17.4
|[YOLOv5m6](https://github.com/ultralytics/yolov5/releases)   |1280 |50.5     |50.5     |68.7     |8.4     | |35.9  |52.4
|[YOLOv5l6](https://github.com/ultralytics/yolov5/releases)   |1280 |53.4     |53.4     |71.1     |12.3    | |77.2  |117.7
|[YOLOv5x6](https://github.com/ultralytics/yolov5/releases)   |1280 |**54.4** |**54.4** |**72.0** |22.4    | |141.8 |222.9
|[YOLOv5x6](https://github.com/ultralytics/yolov5/releases) TTA |1280 |**55.0** |**55.0** |**72.0** |70.8 | |-  |-

<details>
  <summary>Table Notes (click to expand)</summary>

  * AP<sup>test</sup> denotes COCO [test-dev2017](http://cocodataset.org/#upload) server results, all other AP results denote val2017 accuracy.
  * AP values are for single-model single-scale unless otherwise noted. **Reproduce mAP** by `python test.py --data coco.yaml --img 640 --conf 0.001 --iou 0.65`
  * Speed<sub>GPU</sub> averaged over 5000 COCO val2017 images using a GCP [n1-standard-16](https://cloud.google.com/compute/docs/machine-types#n1_standard_machine_types) V100 instance, and includes FP16 inference, postprocessing and NMS. **Reproduce speed** by `python test.py --data coco.yaml --img 640 --conf 0.25 --iou 0.45`
  * All checkpoints are trained to 300 epochs with default settings and hyperparameters (no autoaugmentation).
  * Test Time Augmentation ([TTA](https://github.com/ultralytics/yolov5/issues/303)) includes reflection and scale augmentation. **Reproduce TTA** by `python test.py --data coco.yaml --img 1536 --iou 0.7 --augment`

</details>

<p align="left"><img width="800" src="https://github.com/ultralytics/yolov5/releases/download/v1.0/model_plot.png"></p>

<details>
  <summary>Figure Notes (click to expand)</summary>

  * GPU Speed measures end-to-end time per image averaged over 5000 COCO val2017 images using a V100 GPU with batch size 32, and includes image preprocessing, PyTorch FP16 inference, postprocessing and NMS.
  * EfficientDet data from [google/automl](https://github.com/google/automl) at batch size 8.
  * **Reproduce** by `python test.py --task study --data coco.yaml --iou 0.7 --weights yolov5s6.pt yolov5m6.pt yolov5l6.pt yolov5x6.pt`

</details>

## Load From PyTorch Hub


This example loads a pretrained **YOLOv5s** model and passes an image for inference. YOLOv5 accepts **URL**, **Filename**, **PIL**, **OpenCV**, **Numpy** and **PyTorch** inputs, and returns detections in **torch**, **pandas**, and **JSON** output formats. See our [YOLOv5 PyTorch Hub Tutorial](https://github.com/ultralytics/yolov5/issues/36) for details.

## Citation

[![DOI](https://zenodo.org/badge/264818686.svg)](https://zenodo.org/badge/latestdoi/264818686)


## Inference for video
See  https://github.com/ultralytics/yolov5/issues/36 for more parameters option of YoloV5

In [46]:
from imutils.video import FileVideoStream
from imutils.video import FPS
import numpy as np
import pandas as pd
import argparse
import imutils
import time
import cv2
import matplotlib.pyplot as plt
import torch
from google.colab.patches import cv2_imshow
import pickle

res_name = path+input_name+'.playersbox.yolo'

df_segments = pd.read_csv(path+input_name+'.points.csv', sep=';')

segments = []
for index, row in df_segments.iterrows():
  print(str(row['ini'])+" - "+str(row['fin']+1))
  segments.append(range(row['ini'], row['fin']+1))

# YOLO Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5x', pretrained=True, _verbose=False)
model.cuda()
model.conf = 0.25  # NMS confidence threshold
model.classes = [0]  # (optional list) filter by class, i.e. = [0, 38] for COCO persons, tennis raquet
model.max_det = 50  # maximum number of detections per image
     
video_path = path+input_name+'.mp4'
out_video_root = path+'vis_'+input_name+'.mp4'
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
#videoWriter = cv2.VideoWriter(out_video_root, fourcc, fps, size)
cap.release()

print("Analisis of "+video_path+ " "+str(size)+"x"+str(fps))

print("[INFO] starting video file thread...")
fvs = FileVideoStream(video_path).start()
time.sleep(1.0)
# start the FPS timer
fps = FPS().start()
i = 0
result_box = {}
with open(res_name, "rb") as fp:  
  result_box = pickle.load(fp)

frame_list = []
# loop over frames from the video file stream
while fvs.more():
  frame = fvs.read()
  if any([i in frag for frag in segments]) and i not in result_box:
    results = model(frame)
    #results.save()
    res_list = results.pandas().xyxy[0].values.tolist()
    result_box[i]  = [np.array(r[:-2]) for r in res_list]
      
    if not len(result_box)%500:
      with open(res_name, 'wb') as fp:
        pickle.dump(result_box, fp)
        print(str(i)+": players position exported to "+str(res_name))
  i += 1
  #videoWriter.write(frame)
  if fvs.Q.qsize() < 2:  # If we are low on frames, give time to producer
    time.sleep(0.001)  # Ensures producer runs now, so 2 is sufficient
  fps.update()

# stop the timer and display FPS information
fps.stop()
print("[INFO] elasped time: {:.2f}".format(fps.elapsed()))
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))
fvs.stop()
#videoWriter.release()

with open(res_name, 'wb') as fp:
    pickle.dump(result_box, fp)
    print("End: players position exported to "+str(res_name))


1 - 226
600 - 961
1275 - 2341
2580 - 2596
2895 - 3496
4215 - 4231
4380 - 4501
4710 - 5386
6030 - 6196
6780 - 7051
7455 - 7876
8520 - 8656
8745 - 9241
9660 - 9991
10140 - 10411
10995 - 11071
11265 - 11476
11970 - 12241
12870 - 12886
13215 - 13561
14040 - 14326
14970 - 15181
15345 - 15631
15990 - 16156
16215 - 16921
17880 - 18001


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master


Analisis of /content/gdrive/MyDrive/PadelData/Final_PremierQatar_2_21.mp4 (1920, 1080)x30.0
[INFO] starting video file thread...
12008: players position exported to /content/gdrive/MyDrive/PadelData/Final_PremierQatar_2_21.playersbox.yolo
13466: players position exported to /content/gdrive/MyDrive/PadelData/Final_PremierQatar_2_21.playersbox.yolo
15089: players position exported to /content/gdrive/MyDrive/PadelData/Final_PremierQatar_2_21.playersbox.yolo
16112: players position exported to /content/gdrive/MyDrive/PadelData/Final_PremierQatar_2_21.playersbox.yolo
16671: players position exported to /content/gdrive/MyDrive/PadelData/Final_PremierQatar_2_21.playersbox.yolo
[INFO] elasped time: 579.61
[INFO] approx. FPS: 32.59
End: players position exported to /content/gdrive/MyDrive/PadelData/Final_PremierQatar_2_21.playersbox.yolo


In [30]:
!rm -rf runs/detect/*